In [5]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch_geometric.datasets import Amazon
import torch_geometric.transforms as T
from torch_geometric.nn import GATConv
import numpy as np

dataset = "Photo"
path = osp.join('data', dataset)
dataset = Amazon(path, dataset, transform=T.NormalizeFeatures())
dataset.shuffle()
data = dataset[0]

data = dataset[0]
TRAIN_END = 20 * dataset.num_classes
VAL_END = TRAIN_END + 500
TEST_END = VAL_END + 1000
data = dataset[0]
data.train_mask = torch.zeros([data.num_nodes,], dtype=torch.bool)
data.train_mask[:TRAIN_END] = True
data.val_mask = torch.zeros([data.num_nodes,], dtype=torch.bool)
data.val_mask[TRAIN_END:VAL_END] = True
data.test_mask = torch.zeros([data.num_nodes,], dtype=torch.bool)
data.test_mask[VAL_END:TEST_END] = True


class Net(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Net, self).__init__()

        self.conv1 = GATConv(in_channels, 8, heads=8, dropout=0.6)
        # On the Pubmed dataset, use heads=8 in conv2.
        self.conv2 = GATConv(8 * 8, out_channels, heads=1, concat=False,
                             dropout=0.6)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=-1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dataset.num_features, dataset.num_classes).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)


def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()


@torch.no_grad()
def test(data):
    model.eval()
    out, accs = model(data.x, data.edge_index), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        acc = float((out[mask].argmax(-1) == data.y[mask]).sum() / mask.sum())
        accs.append(acc)
    return accs


print()
avg = np.empty([1,])
max_val = 0
early_stop_cnt = 0
for epoch in range(1, 1001):
    train(data)
    train_acc, val_acc, test_acc = test(data)

    if val_acc > max_val:
        early_stop_cnt = 0
        max_val = val_acc
    else:
        early_stop_cnt += 1
        if (early_stop_cnt == 100):
            print(f'Early Stopping at epoch {epoch:03d}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')
            break

    avg = np.append(avg, test_acc)
    last_hun = np.average(avg[len(avg)-100:])

    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, '
          f'Test: {test_acc:.4f}, Avg100: {last_hun:.4f}')


Epoch: 001, Train: 0.4563, Val: 0.4540, Test: 0.4050, Avg100: 32.4120
Epoch: 002, Train: 0.3500, Val: 0.3500, Test: 0.3200, Avg100: 21.7147
Epoch: 003, Train: 0.2875, Val: 0.2280, Test: 0.2100, Avg100: 16.3385
Epoch: 004, Train: 0.2875, Val: 0.2260, Test: 0.2100, Avg100: 13.1128
Epoch: 005, Train: 0.2688, Val: 0.2220, Test: 0.2030, Avg100: 10.9612
Epoch: 006, Train: 0.2688, Val: 0.2220, Test: 0.2030, Avg100: 9.4243
Epoch: 007, Train: 0.2688, Val: 0.2220, Test: 0.2020, Avg100: 8.2715
Epoch: 008, Train: 0.2688, Val: 0.2220, Test: 0.2020, Avg100: 7.3749
Epoch: 009, Train: 0.2688, Val: 0.2220, Test: 0.2020, Avg100: 6.6576
Epoch: 010, Train: 0.2688, Val: 0.2220, Test: 0.2020, Avg100: 6.0707
Epoch: 011, Train: 0.2688, Val: 0.2220, Test: 0.2020, Avg100: 5.5817
Epoch: 012, Train: 0.2688, Val: 0.2220, Test: 0.2020, Avg100: 5.1678
Epoch: 013, Train: 0.2688, Val: 0.2220, Test: 0.2020, Avg100: 4.8131
Epoch: 014, Train: 0.2688, Val: 0.2220, Test: 0.2020, Avg100: 4.5057
Epoch: 015, Train: 0.2688, V

Epoch: 124, Train: 0.7500, Val: 0.6840, Test: 0.6230, Avg100: 0.4992
Epoch: 125, Train: 0.7500, Val: 0.6800, Test: 0.6260, Avg100: 0.5013
Epoch: 126, Train: 0.7563, Val: 0.6780, Test: 0.6250, Avg100: 0.5035
Epoch: 127, Train: 0.7563, Val: 0.6700, Test: 0.6240, Avg100: 0.5057
Epoch: 128, Train: 0.7375, Val: 0.6480, Test: 0.6110, Avg100: 0.5078
Epoch: 129, Train: 0.7437, Val: 0.6440, Test: 0.6010, Avg100: 0.5100
Epoch: 130, Train: 0.7437, Val: 0.6440, Test: 0.6020, Avg100: 0.5124
Epoch: 131, Train: 0.7437, Val: 0.6460, Test: 0.6030, Avg100: 0.5151
Epoch: 132, Train: 0.7563, Val: 0.6540, Test: 0.6180, Avg100: 0.5180
Epoch: 133, Train: 0.7688, Val: 0.6740, Test: 0.6250, Avg100: 0.5206
Epoch: 134, Train: 0.7625, Val: 0.6760, Test: 0.6290, Avg100: 0.5233
Epoch: 135, Train: 0.7625, Val: 0.6760, Test: 0.6300, Avg100: 0.5258
Epoch: 136, Train: 0.7688, Val: 0.6780, Test: 0.6290, Avg100: 0.5281
Epoch: 137, Train: 0.7688, Val: 0.6800, Test: 0.6330, Avg100: 0.5304
Epoch: 138, Train: 0.7688, Val: 0.

Epoch: 250, Train: 0.7750, Val: 0.6720, Test: 0.6240, Avg100: 0.6260
Epoch: 251, Train: 0.7750, Val: 0.6620, Test: 0.6200, Avg100: 0.6261
Epoch: 252, Train: 0.7625, Val: 0.6580, Test: 0.6170, Avg100: 0.6260
Epoch: 253, Train: 0.7750, Val: 0.6660, Test: 0.6270, Avg100: 0.6261
Epoch: 254, Train: 0.7875, Val: 0.6740, Test: 0.6360, Avg100: 0.6263
Epoch: 255, Train: 0.7875, Val: 0.6820, Test: 0.6320, Avg100: 0.6264
Epoch: 256, Train: 0.7875, Val: 0.6900, Test: 0.6350, Avg100: 0.6264
Epoch: 257, Train: 0.7750, Val: 0.6920, Test: 0.6360, Avg100: 0.6265
Epoch: 258, Train: 0.7750, Val: 0.6840, Test: 0.6330, Avg100: 0.6266
Epoch: 259, Train: 0.7875, Val: 0.6940, Test: 0.6520, Avg100: 0.6269
Epoch: 260, Train: 0.8188, Val: 0.7400, Test: 0.6930, Avg100: 0.6275
Epoch: 261, Train: 0.8188, Val: 0.7460, Test: 0.6970, Avg100: 0.6282
Epoch: 262, Train: 0.8250, Val: 0.7520, Test: 0.7020, Avg100: 0.6289
Epoch: 263, Train: 0.8125, Val: 0.7200, Test: 0.6680, Avg100: 0.6294
Epoch: 264, Train: 0.7875, Val: 0.

Epoch: 376, Train: 0.8250, Val: 0.7780, Test: 0.7220, Avg100: 0.6725
Epoch: 377, Train: 0.7937, Val: 0.7160, Test: 0.6750, Avg100: 0.6727
Epoch: 378, Train: 0.7937, Val: 0.7100, Test: 0.6680, Avg100: 0.6729
Epoch: 379, Train: 0.7937, Val: 0.7120, Test: 0.6690, Avg100: 0.6732
Epoch: 380, Train: 0.8125, Val: 0.7320, Test: 0.6950, Avg100: 0.6738
Epoch: 381, Train: 0.8313, Val: 0.7680, Test: 0.7130, Avg100: 0.6746
Epoch: 382, Train: 0.8375, Val: 0.7800, Test: 0.7270, Avg100: 0.6754
Epoch: 383, Train: 0.8438, Val: 0.7820, Test: 0.7270, Avg100: 0.6762
Epoch: 384, Train: 0.8375, Val: 0.7700, Test: 0.7230, Avg100: 0.6768
Epoch: 385, Train: 0.8313, Val: 0.7640, Test: 0.7160, Avg100: 0.6774
Epoch: 386, Train: 0.8313, Val: 0.7660, Test: 0.7150, Avg100: 0.6781
Epoch: 387, Train: 0.8250, Val: 0.7640, Test: 0.7150, Avg100: 0.6789
Epoch: 388, Train: 0.8188, Val: 0.7480, Test: 0.7060, Avg100: 0.6795
Epoch: 389, Train: 0.8313, Val: 0.7800, Test: 0.7200, Avg100: 0.6800
Epoch: 390, Train: 0.8375, Val: 0.

Epoch: 501, Train: 0.8875, Val: 0.8300, Test: 0.7780, Avg100: 0.7413
Epoch: 502, Train: 0.8813, Val: 0.8360, Test: 0.7780, Avg100: 0.7419
Epoch: 503, Train: 0.8562, Val: 0.8160, Test: 0.7620, Avg100: 0.7423
Epoch: 504, Train: 0.8500, Val: 0.7940, Test: 0.7530, Avg100: 0.7428
Epoch: 505, Train: 0.8500, Val: 0.7900, Test: 0.7430, Avg100: 0.7432
Epoch: 506, Train: 0.8313, Val: 0.7640, Test: 0.7240, Avg100: 0.7432
Epoch: 507, Train: 0.8313, Val: 0.7640, Test: 0.7250, Avg100: 0.7429
Epoch: 508, Train: 0.8438, Val: 0.7980, Test: 0.7570, Avg100: 0.7429
Epoch: 509, Train: 0.8562, Val: 0.8100, Test: 0.7610, Avg100: 0.7430
Epoch: 510, Train: 0.8562, Val: 0.8180, Test: 0.7680, Avg100: 0.7434
Epoch: 511, Train: 0.8562, Val: 0.8180, Test: 0.7610, Avg100: 0.7440
Epoch: 512, Train: 0.8562, Val: 0.8160, Test: 0.7610, Avg100: 0.7449
Epoch: 513, Train: 0.8438, Val: 0.8020, Test: 0.7540, Avg100: 0.7457
Epoch: 514, Train: 0.8438, Val: 0.7880, Test: 0.7470, Avg100: 0.7462
Epoch: 515, Train: 0.8313, Val: 0.

Epoch: 623, Train: 0.8625, Val: 0.8120, Test: 0.7710, Avg100: 0.7509
Epoch: 624, Train: 0.8562, Val: 0.8140, Test: 0.7670, Avg100: 0.7510
Epoch: 625, Train: 0.8562, Val: 0.8080, Test: 0.7590, Avg100: 0.7510
Epoch: 626, Train: 0.8438, Val: 0.7960, Test: 0.7510, Avg100: 0.7511
Epoch: 627, Train: 0.8375, Val: 0.7960, Test: 0.7490, Avg100: 0.7511
Epoch: 628, Train: 0.8500, Val: 0.8020, Test: 0.7520, Avg100: 0.7511
Epoch: 629, Train: 0.8625, Val: 0.8100, Test: 0.7620, Avg100: 0.7512
Epoch: 630, Train: 0.8625, Val: 0.8060, Test: 0.7600, Avg100: 0.7512
Epoch: 631, Train: 0.8500, Val: 0.7980, Test: 0.7570, Avg100: 0.7511
Epoch: 632, Train: 0.8500, Val: 0.7940, Test: 0.7490, Avg100: 0.7510
Epoch: 633, Train: 0.8500, Val: 0.7840, Test: 0.7480, Avg100: 0.7510
Epoch: 634, Train: 0.8562, Val: 0.7780, Test: 0.7430, Avg100: 0.7510
Epoch: 635, Train: 0.8500, Val: 0.7840, Test: 0.7420, Avg100: 0.7510
Epoch: 636, Train: 0.8562, Val: 0.7860, Test: 0.7490, Avg100: 0.7512
Epoch: 637, Train: 0.8625, Val: 0.